## Classification with CNN(MNIST)



In [1]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

C:\Users\user\Anaconda3\lib\site-packages\wheel\pep425tags.py:75: RuntimeWarning: Config variable 'Py_DEBUG' is unset, Python ABI tag may be incorrect
  warn=(impl == 'cp')):
C:\Users\user\Anaconda3\lib\site-packages\wheel\pep425tags.py:79: RuntimeWarning: Config variable 'WITH_PYMALLOC' is unset, Python ABI tag may be incorrect
  warn=(impl == 'cp')):
torch-0.4.1-cp36-cp36m-linux_x86_64.whl is not a supported wheel on this platform.
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
print(torch.__version__)
print(torch.cuda.is_available())

1.0.0
False


In [3]:

import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [5]:
# torch.device()를 통해 gpu를 이용합니다. 
# cuda:뒤의 번호는 gpu를 여러개 동시 사용할 때 쓰이는 gpu 번호입니다.
# gpu 사용이 가능한 경우 torch.cuda.is_available() = True이고 아닌 경우 False가 됩니다.

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [6]:
# hyper parameter들을 설정합니다.

num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [7]:
# MNIST dataset을 불러옵니다.

train_dataset = torchvision.datasets.MNIST(root='./data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

Processing...
Done!


In [8]:
# dataloader를 이용해 batch_size만큼 데이터를 불러올 수 있도록 합니다.

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

## Modeling

In [15]:
class ConvNet(nn.Module):
    def __init__(self,num_classes = 10):
        super(ConvNet, self).__init__()
        
        # conv2d 에서는 28X28 그대로 나온다
        # pooling을 통해 size는 14 X 14가 된다
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,16,kernel_size=5, stride = 1, padding = 2),
            nn.BatchNorm2d(num_features = 16), ## batchnorm은 activation function 전에 넣는다
            nn.ReLU(),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride = 2)
            )
        
        # conv2d에서는 14 X 14 그대로 나옵니다. 
        self.layer2 = nn.Sequential(
            nn.Conv2d(16,32,kernel_size = 5, stride = 1, padding = 2),
            nn.BatchNorm2d(num_features = 32), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
            )
        
        self.fc = nn.Linear(7*7*32,num_classes)
        
    def forward(self,x):
        x = self.layer1(x)
        x = self.layer2(x)
        
        # flatten
        x = x.reshape(x.size(0),-1) # batchsize를 제외한 나머지 
        out = self.fc(x)
        return out
    
model = ConvNet(num_classes).to(device)
        

## Loss & optimizer

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

## Training

In [17]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader): ## 데이터 600개에서 100새개씩
        images = images.to(device)                      ## i는 인덱스(0 ~ 600)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}],Loss: {:.4f}'
                  .format(epoch+1,num_epochs, i+1, total_step,loss.item()))

Epoch [1/5], Step [100/600],Loss: 0.1244
Epoch [1/5], Step [200/600],Loss: 0.1054
Epoch [1/5], Step [300/600],Loss: 0.1559
Epoch [1/5], Step [400/600],Loss: 0.1193
Epoch [1/5], Step [500/600],Loss: 0.0897
Epoch [1/5], Step [600/600],Loss: 0.0701
Epoch [2/5], Step [100/600],Loss: 0.0504
Epoch [2/5], Step [200/600],Loss: 0.0898
Epoch [2/5], Step [300/600],Loss: 0.0318
Epoch [2/5], Step [400/600],Loss: 0.0472
Epoch [2/5], Step [500/600],Loss: 0.0389
Epoch [2/5], Step [600/600],Loss: 0.0464
Epoch [3/5], Step [100/600],Loss: 0.0104
Epoch [3/5], Step [200/600],Loss: 0.0581
Epoch [3/5], Step [300/600],Loss: 0.0576
Epoch [3/5], Step [400/600],Loss: 0.0247
Epoch [3/5], Step [500/600],Loss: 0.0747
Epoch [3/5], Step [600/600],Loss: 0.0930
Epoch [4/5], Step [100/600],Loss: 0.0242
Epoch [4/5], Step [200/600],Loss: 0.0401
Epoch [4/5], Step [300/600],Loss: 0.0279
Epoch [4/5], Step [400/600],Loss: 0.0121
Epoch [4/5], Step [500/600],Loss: 0.0283
Epoch [4/5], Step [600/600],Loss: 0.0126
Epoch [5/5], Ste

## Evaluation

In [19]:
# eval() 에서는 dropout, batchnorm등의 동작이 다릅니다.
# dropout은 작동되지 않고 batchnorm의 running_mean, running_std는 normalization을 위해 사용 됩니다.
# 자세한 내용은 torch.nn.BatchNorm2d()을 참조
# torch.no_grad() autograd가 작동하지 않도록 해서 메모리를 줄이고 속도를 높인다

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        _,predicted = torch.max(outputs.data,1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))
    
    

Test Accuracy of the model on the 10000 test images: 98.82 %
